# GloVe Embeddings using Keras
## This notebook outlines the concepts of using pre-trained GloVe Embeddings using Keras

### Import the necessary libraries

In [11]:
from numpy import array, asarray, zeros
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

### Define the corpus

In [30]:
docs = ['Well 2 done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

### Define the labels

In [31]:
labels = array([1,1,1,1,1,0,0,0,0,0])

### Tokenizer

In [32]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

### Integer encode the documents

In [33]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 7, 2], [3, 1], [8, 4], [9, 1], [10], [11], [5, 4], [12, 3], [5, 1], [13, 14, 2, 15]]


### Padding documents to a desired max_length

In [6]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


### Load the GloVe Embedding into memory

In [8]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.50d.txt')
# f = open('glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    print(word)
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

the
,
.
of
to
and
in
a
"
's
for
-
that
on
is
was
said
with
he
as
it
by
at
(
)
from
his
''
``
an
be
has
are
have
but
were
not
this
who
they
had
i
which
will
their
:
or
its
one
after
new
been
also
we
would
two
more
'
first
about
up
when
year
there
all
--
out
she
other
people
n't
her
percent
than
over
into
last
some
government
time
$
you
years
if
no
world
can
three
do
;
president
only
state
million
could
us
most
_
against
u.s.
so
them
what
him
united
during
before
may
since
many
while
where
states
because
now
city
made
like
between
did
just
national
day
country
under
such
second
then
company
group
any
through
china
four
being
down
war
back
off
south
american
minister
police
well
including
team
international
week
officials
still
both
even
high
part
told
those
end
former
these
make
billion
work
our
home
school
party
house
old
later
get
another
tuesday
news
long
five
called
1
wednesday
military
way
used
much
next
monday
thursday
friday
game
here
?
should
take
very
my
north
security
season
yo

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2273: character maps to <undefined>

### Create Embedding matrix for the custom dataset

In [9]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Define the Embedding layer

In [12]:
e = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=4, trainable=False)

### Define the model

In [23]:
model = Sequential()
model.add(e)
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Compiling the model

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### Summarize the model

In [25]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 50)             750       
_________________________________________________________________
flatten_2 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                4020      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 4,991
Trainable params: 4,241
Non-trainable params: 750
_________________________________________________________________
None


### Build the model

In [28]:
model.fit(padded_docs, labels, epochs=50)

Train on 10 samples
Epoch 1/50
10/10 [==============================] - 0s 601us/sample - loss: 0.1733 - acc: 0.9000
Epoch 2/50
10/10 [==============================] - 0s 300us/sample - loss: 0.1724 - acc: 0.9000
Epoch 3/50
10/10 [==============================] - 0s 350us/sample - loss: 0.1715 - acc: 0.9000
Epoch 4/50
10/10 [==============================] - 0s 299us/sample - loss: 0.1706 - acc: 0.9000
Epoch 5/50
10/10 [==============================] - 0s 200us/sample - loss: 0.1698 - acc: 0.9000
Epoch 6/50
10/10 [==============================] - 0s 300us/sample - loss: 0.1690 - acc: 0.9000
Epoch 7/50
10/10 [==============================] - 0s 200us/sample - loss: 0.1682 - acc: 0.9000
Epoch 8/50
10/10 [==============================] - 0s 253us/sample - loss: 0.1675 - acc: 0.9000
Epoch 9/50
10/10 [==============================] - 0s 300us/sample - loss: 0.1668 - acc: 0.9000
Epoch 10/50
10/10 [==============================] - 0s 300us/sample - loss: 0.1661 - acc: 0.9000
Epoch 11/

### Evaluating the model

In [29]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998
